# Libraries

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

# Preprocessing

In [2]:
# base path
directory_path = '/mnt/d/GitHub/SSLrecognition/train_data/videos'
# current directory
c_dir = os.getcwd()

# all actions
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

# specific actions
# actions = np.array(['base', 'again', 'how', 'open', 'sorry', 'see'])

In [3]:
# create a dictionary for int representation of actions
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'afternoon': 0,
 'again': 1,
 'base': 2,
 'door': 3,
 'hello': 4,
 'house': 5,
 'how': 6,
 'open': 7,
 'see': 8,
 'sorry': 9,
 'why': 10}

Note that at this point, we will not access the video folder, only the numpy folder.

In [4]:
sequences, labels = [], []  # sequence -> video, labels -> action
for action in actions:
    no_actions = len(os.listdir(os.path.join(c_dir, 'labels', action)))
    print('Opening path:', os.path.join(c_dir, 'labels', action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []         # window -> single frame
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(c_dir, 'labels', action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        for frame_num in range(1, no_frames_per_action + 1):
            res = np.load(os.path.join(c_dir, 'labels', action, file,  "{}.npy".format(frame_num)))     # res -> coordinate key points
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    print('-'*75)

Opening path: /mnt/d/GitHub/SSLrecognition/train_data/labels/afternoon
Number of instances: 40
Number of frames in afternoon_1: 31
Number of frames in afternoon_2: 30
Number of frames in afternoon_3: 30
Number of frames in afternoon_4: 30
Number of frames in afternoon_5: 31
Number of frames in afternoon_6: 31
Number of frames in afternoon_7: 31
Number of frames in afternoon_8: 30
Number of frames in afternoon_9: 31
Number of frames in afternoon_10: 31
Number of frames in afternoon_11: 31
Number of frames in afternoon_12: 31
Number of frames in afternoon_13: 31
Number of frames in afternoon_14: 31
Number of frames in afternoon_15: 31
Number of frames in afternoon_16: 31
Number of frames in afternoon_17: 31
Number of frames in afternoon_18: 31
Number of frames in afternoon_19: 31
Number of frames in afternoon_20: 31
Number of frames in afternoon_21: 31
Number of frames in afternoon_22: 31
Number of frames in afternoon_23: 31
Number of frames in afternoon_24: 31
Number of frames in aftern

In [5]:
# due to difference in number of frames, pad x and y
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = pad_sequences(to_categorical(labels).astype(int), dtype = 'int', padding = 'post', value = -1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

# Models

## LSTM

In [284]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN, EarlyStopping

In [285]:
# for logging of data with TensorBoard
log_dir = os.path.join(c_dir, 'Logs')
tb_callback = TensorBoard(log_dir = log_dir)

# to end training when failure happens ie. loss == nan
term = TerminateOnNaN()

# to stop training early if there is no change in loss
early = EarlyStopping(monitor = 'loss', patience = 5)

In [286]:
x_train.shape

(216, 117, 225)

In [287]:
# model = Sequential()
# model.add(LSTM(64, return_sequences = True, activation = "relu", input_shape = (117, 225)))
# model.add(LSTM(128, return_sequences = True, activation = "relu"))
# model.add(LSTM(64, return_sequences = False, activation = "relu"))
# model.add(Dense(64, activation = "relu"))
# model.add(Dense(32, activation = "relu"))
# model.add(Dense(8, activation = "relu"))
# model.add(Dense(actions.shape[0], activation = "softmax"))

In [288]:
# model = Sequential()
# model.add(LSTM(64, input_shape = (117, 225), return_sequences = True))
# model.add(Dropout(0.2))
# model.add(LSTM(128, return_sequences = True))
# model.add(Dropout(0.2))
# model.add(LSTM(32))
# model.add(Dropout(0.2))
# model.add(Dense(16, activation = 'relu'))
# model.add(Dense(actions.shape[0], activation = "softmax"))

In [289]:
model = Sequential()
model.add(LSTM(64, input_shape = (117, 225)))
model.add(Dense(64, activation = "relu"))
model.add(Dense(actions.shape[0], activation = "softmax"))

In [290]:
opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_66 (LSTM)              (None, 64)                74240     
                                                                 
 dense_39 (Dense)            (None, 64)                4160      
                                                                 
 dense_40 (Dense)            (None, 11)                715       
                                                                 
Total params: 79115 (309.04 KB)
Trainable params: 79115 (309.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [291]:
model.fit(x_train, y_train, epochs = 2000, callbacks = [tb_callback, term, early])

Epoch 1/2000


7/7 [==============================] - 1s 47ms/step - loss: 2.2691 - categorical_accuracy: 0.1389
Epoch 2/2000
7/7 [==============================] - 0s 14ms/step - loss: 2.0994 - categorical_accuracy: 0.1759
Epoch 3/2000
7/7 [==============================] - 0s 16ms/step - loss: 2.3318 - categorical_accuracy: 0.1806
Epoch 4/2000
7/7 [==============================] - 0s 16ms/step - loss: 2.0176 - categorical_accuracy: 0.1620
Epoch 5/2000
7/7 [==============================] - 0s 15ms/step - loss: 2.0046 - categorical_accuracy: 0.1852
Epoch 6/2000
7/7 [==============================] - 0s 15ms/step - loss: 1.9369 - categorical_accuracy: 0.1944
Epoch 7/2000
7/7 [==============================] - 0s 15ms/step - loss: 1.9349 - categorical_accuracy: 0.1667
Epoch 8/2000
7/7 [==============================] - 0s 14ms/step - loss: 1.9525 - categorical_accuracy: 0.1667
Epoch 9/2000
7/7 [==============================] - 0s 14ms/step - loss: 1.9147 - categorical_accuracy: 0.1898
Epoch 10/2000


In [292]:
# take model predictions
res = model.predict(x_test)
res

1/1 [==============================] - 0s 221ms/step


array([[3.79325539e-01, 7.03423633e-04, 8.27647373e-02, 1.11517664e-02,
        1.72522664e-01, 7.25844060e-04, 1.33320987e-01, 3.58821679e-04,
        6.81954250e-02, 1.04354031e-01, 4.65767868e-02],
       [1.97364613e-01, 2.69043550e-04, 1.87952101e-01, 1.12036976e-03,
        4.06936742e-03, 1.32708214e-04, 6.03104472e-01, 4.88689802e-05,
        2.29494739e-03, 1.91429816e-03, 1.72919082e-03],
       [3.48281232e-03, 1.59441016e-03, 9.71186906e-03, 1.76633727e-02,
        2.71123618e-01, 3.37592489e-03, 2.26248056e-03, 1.45879318e-03,
        4.49052930e-01, 9.67783704e-02, 1.43495411e-01],
       [5.23732841e-01, 3.81312158e-04, 1.24811582e-01, 4.29213559e-03,
        3.79701369e-02, 2.61496287e-04, 2.50495434e-01, 1.61040371e-04,
        1.88869275e-02, 2.64474675e-02, 1.25595825e-02],
       [4.51587588e-01, 1.49253523e-04, 1.39433846e-01, 1.38207851e-03,
        9.61113255e-03, 7.96610329e-05, 3.82417768e-01, 4.74581684e-05,
        5.65163186e-03, 6.40374096e-03, 3.23579670e-

In [293]:
y_pred = (res == res.max(axis=1, keepdims=True)).astype(int)
print(y_pred)
print(y_test)

[[1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]

In [294]:
correct_rows = np.sum(np.argmax(y_pred, axis=1) == np.argmax(y_test, axis=1))
total_rows = y_pred.shape[0]
accuracy = correct_rows / total_rows * 100
print(f'Accuracy: {round(accuracy, 3)}%')

Accuracy: 66.667%
